In [1]:
import doctest
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.datasets import make_blobs
from scipy.stats import multivariate_normal
from matplotlib.patches import Ellipse

# two useful data viz libraries
import matplotlib.pyplot as plt
import seaborn as sns
# setup plotting in a notebook in a reasonable way
%matplotlib inline
%config InlineBackend.figure_format='retina'

# default figure aesthetics I'll be using, 
# there are other choices, see seaborn docs
sns.set_style("white")
sns.set_context("notebook")

In [2]:
zillow = pd.read_csv(r"C:\Users\krist\Cogs 118B project demo\cleaned_df.csv\cleaned_df.csv")
zillow

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,RentEstimate,Latitude,Longitude,ListedPrice
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.3805,240600.0,1599.0,30.819534,-88.095960,239900.0
1,AL,Robertsdale,Cowpen Creek Rd,36567.0,3.0,2.0,1800.0,144.388889,3.2000,NaN,NaN,30.590004,-87.580376,259900.0
2,AL,Gulf Shores,Spinnaker Dr #201,36542.0,2.0,2.0,1250.0,274.000000,NaN,NaN,NaN,30.284956,-87.747920,342500.0
3,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.2600,336200.0,1932.0,33.357986,-86.608700,335000.0
4,AL,Huntsville,Turtlebrook Ct,35811.0,3.0,2.0,1225.0,204.081633,NaN,222700.0,1679.0,34.775517,-86.440700,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22676,WY,Burns,Road 210a,82053.0,2.0,1.0,1112.0,337.230216,20.0000,368600.0,1500.0,41.149517,-104.439620,375000.0
22677,WY,Cody,Lincoln Ave,82414.0,2.0,3.0,1232.0,241.801948,0.0300,NaN,NaN,44.524570,-109.033844,297900.0
22678,WY,Dubois,Mason Dr,82513.0,2.0,2.0,2016.0,262.896825,1.4200,NaN,2294.0,43.521640,-109.594050,530000.0
22679,WY,Cheyenne,Carolina Rd,82009.0,3.0,2.0,2930.0,196.552901,6.2300,558000.0,2749.0,41.138450,-105.052790,575900.0


In [3]:
zillow['LotArea'][2]

nan

In [4]:
zillow = zillow.dropna()
zillow

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,RentEstimate,Latitude,Longitude,ListedPrice
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.380500,240600.0,1599.0,30.819534,-88.095960,239900.0
3,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.260000,336200.0,1932.0,33.357986,-86.608700,335000.0
5,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.200000,150500.0,1385.0,32.372746,-86.165115,151000.0
6,AL,Boaz,Greenwood Ave,35957.0,3.0,2.0,1717.0,139.196273,0.380000,238400.0,2125.0,34.210014,-86.136690,239000.0
7,AL,Albertville,Lexington Ave,35950.0,3.0,2.0,1674.0,149.283154,0.344353,248000.0,1597.0,34.275400,-86.217920,249900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22674,WY,Douglas,Bar Two Dr,82633.0,3.0,3.0,2316.0,133.851468,0.150000,306700.0,1724.0,42.756350,-105.399580,310000.0
22675,WY,Riverton,Riverview Rd,82501.0,3.0,3.0,2400.0,77.083333,3.350000,184500.0,1999.0,43.077744,-108.558960,185000.0
22676,WY,Burns,Road 210a,82053.0,2.0,1.0,1112.0,337.230216,20.000000,368600.0,1500.0,41.149517,-104.439620,375000.0
22679,WY,Cheyenne,Carolina Rd,82009.0,3.0,2.0,2930.0,196.552901,6.230000,558000.0,2749.0,41.138450,-105.052790,575900.0


In [7]:
zillow.columns

Index(['State', 'City', 'Street', 'Zipcode', 'Bedroom', 'Bathroom', 'Area',
       'PPSq', 'LotArea', 'MarketEstimate', 'RentEstimate', 'Latitude',
       'Longitude', 'ListedPrice'],
      dtype='object')

In [11]:
type_list = []
for variable in zillow.columns:
    type_list = np.append(type_list, type(zillow[variable][0]))
type_list

array([<class 'str'>, <class 'str'>, <class 'str'>,
       <class 'numpy.float64'>, <class 'numpy.float64'>,
       <class 'numpy.float64'>, <class 'numpy.float64'>,
       <class 'numpy.float64'>, <class 'numpy.float64'>,
       <class 'numpy.float64'>, <class 'numpy.float64'>,
       <class 'numpy.float64'>, <class 'numpy.float64'>,
       <class 'numpy.float64'>], dtype=object)

In [12]:
city_df = pd.read_csv(r"C:\Users\krist\Cogs 118B project demo\simplemaps_uscities_basicv1.78\uscities.csv")
city_df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18908608,11080.3,shape,False,True,America/New_York,1,11229 11228 11226 11225 11224 11222 11221 1122...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,11922389,3184.7,shape,False,True,America/Los_Angeles,1,91367 90291 90293 90292 91316 91311 90035 9003...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8497759,4614.5,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,6080145,4758.9,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149
4,Houston,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,5970127,1384.0,shape,False,True,America/Chicago,1,77069 77068 77061 77060 77063 77062 77065 7706...,1840020925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31115,Kohatk,Kohatk,AZ,Arizona,4021,Pinal,32.5781,-112.0032,0,77.3,shape,False,False,America/Phoenix,3,85634,1840022983
31116,Ironville,Ironville,PA,Pennsylvania,42013,Blair,40.6586,-78.2155,0,59.2,shape,False,False,America/New_York,3,16686,1840152922
31117,Newkirk,Newkirk,NM,New Mexico,35019,Guadalupe,35.0635,-104.2715,0,0.0,shape,False,False,America/Denver,3,88417,1840024978
31118,Falcon Village,Falcon Village,TX,Texas,48427,Starr,26.5652,-99.1341,0,38.9,shape,False,False,America/Chicago,3,78545,1840018314


In [15]:
len(city_df['city'].unique())

20855

In [13]:
city_df.columns

Index(['city', 'city_ascii', 'state_id', 'state_name', 'county_fips',
       'county_name', 'lat', 'lng', 'population', 'density', 'source',
       'military', 'incorporated', 'timezone', 'ranking', 'zips', 'id'],
      dtype='object')

In [22]:
appended_df = zillow.merge(city_df, left_on = ['City', 'State'], right_on = ['city', 'state_id'])
appended_df

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.3805,240600.0,...,-88.1004,16109,193.5,shape,False,True,America/Chicago,3,36571,1840015886
1,AL,Saraland,Deer Run Dr,36571.0,3.0,2.0,1828.0,108.315098,0.4658,199700.0,...,-88.1004,16109,193.5,shape,False,True,America/Chicago,3,36571,1840015886
2,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.2600,336200.0,...,-86.6300,15201,181.7,shape,False,True,America/Chicago,3,35051 35043,1840014823
3,AL,Chelsea,Highway 440,35043.0,3.0,2.0,1425.0,195.789474,1.8200,280200.0,...,-86.6300,15201,181.7,shape,False,True,America/Chicago,3,35051 35043,1840014823
4,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.2000,150500.0,...,-86.2668,251158,482.6,shape,False,True,America/Chicago,2,36112 36113 36110 36111 36116 36114 36115 3610...,1840008353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12599,WY,Big Piney,Taylor Ave,83113.0,4.0,2.0,1820.0,112.637363,0.1600,189800.0,...,-110.1193,450,342.1,shape,False,True,America/Denver,3,83113,1840020061
12600,WY,Daniel,Booth Dr,83115.0,1.0,1.0,1008.0,337.202381,10.0600,307500.0,...,-110.0766,158,12.0,shape,False,False,America/Denver,3,83115,1840018659
12601,WY,Shoshoni,Pine St,82649.0,6.0,3.0,3294.0,74.377656,0.3200,236800.0,...,-108.1068,640,69.0,shape,False,True,America/Denver,3,82649,1840022355
12602,WY,Mills,N 4th Ave,82604.0,4.0,2.0,2120.0,90.094340,0.3100,184600.0,...,-106.3830,4218,319.8,shape,False,True,America/Denver,3,82644 82604 82601,1840022359


In [23]:
appended_df.columns

Index(['State', 'City', 'Street', 'Zipcode', 'Bedroom', 'Bathroom', 'Area',
       'PPSq', 'LotArea', 'MarketEstimate', 'RentEstimate', 'Latitude',
       'Longitude', 'ListedPrice', 'city', 'city_ascii', 'state_id',
       'state_name', 'county_fips', 'county_name', 'lat', 'lng', 'population',
       'density', 'source', 'military', 'incorporated', 'timezone', 'ranking',
       'zips', 'id'],
      dtype='object')

In [24]:
appended_df = appended_df.drop(columns = ['city','city_ascii','state_id',
       'state_name', 'county_fips', 'county_name', 'lat', 'lng'])

In [25]:
appended_df.columns

Index(['State', 'City', 'Street', 'Zipcode', 'Bedroom', 'Bathroom', 'Area',
       'PPSq', 'LotArea', 'MarketEstimate', 'RentEstimate', 'Latitude',
       'Longitude', 'ListedPrice', 'population', 'density', 'source',
       'military', 'incorporated', 'timezone', 'ranking', 'zips', 'id'],
      dtype='object')

In [26]:
appended_df

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,ListedPrice,population,density,source,military,incorporated,timezone,ranking,zips,id
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.3805,240600.0,...,239900.0,16109,193.5,shape,False,True,America/Chicago,3,36571,1840015886
1,AL,Saraland,Deer Run Dr,36571.0,3.0,2.0,1828.0,108.315098,0.4658,199700.0,...,198000.0,16109,193.5,shape,False,True,America/Chicago,3,36571,1840015886
2,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.2600,336200.0,...,335000.0,15201,181.7,shape,False,True,America/Chicago,3,35051 35043,1840014823
3,AL,Chelsea,Highway 440,35043.0,3.0,2.0,1425.0,195.789474,1.8200,280200.0,...,279000.0,15201,181.7,shape,False,True,America/Chicago,3,35051 35043,1840014823
4,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.2000,150500.0,...,151000.0,251158,482.6,shape,False,True,America/Chicago,2,36112 36113 36110 36111 36116 36114 36115 3610...,1840008353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12599,WY,Big Piney,Taylor Ave,83113.0,4.0,2.0,1820.0,112.637363,0.1600,189800.0,...,205000.0,450,342.1,shape,False,True,America/Denver,3,83113,1840020061
12600,WY,Daniel,Booth Dr,83115.0,1.0,1.0,1008.0,337.202381,10.0600,307500.0,...,339900.0,158,12.0,shape,False,False,America/Denver,3,83115,1840018659
12601,WY,Shoshoni,Pine St,82649.0,6.0,3.0,3294.0,74.377656,0.3200,236800.0,...,245000.0,640,69.0,shape,False,True,America/Denver,3,82649,1840022355
12602,WY,Mills,N 4th Ave,82604.0,4.0,2.0,2120.0,90.094340,0.3100,184600.0,...,191000.0,4218,319.8,shape,False,True,America/Denver,3,82644 82604 82601,1840022359


In [27]:
appended_df['source'].unique()

array(['shape', 'point'], dtype=object)

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,ListedPrice,population,density,source,military,incorporated,timezone,ranking,zips,id
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.3805,240600.0,...,239900.0,16109,193.5,shape,False,True,America/Chicago,3,36571,1840015886
1,AL,Saraland,Deer Run Dr,36571.0,3.0,2.0,1828.0,108.315098,0.4658,199700.0,...,198000.0,16109,193.5,shape,False,True,America/Chicago,3,36571,1840015886
2,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.2600,336200.0,...,335000.0,15201,181.7,shape,False,True,America/Chicago,3,35051 35043,1840014823
3,AL,Chelsea,Highway 440,35043.0,3.0,2.0,1425.0,195.789474,1.8200,280200.0,...,279000.0,15201,181.7,shape,False,True,America/Chicago,3,35051 35043,1840014823
4,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.2000,150500.0,...,151000.0,251158,482.6,shape,False,True,America/Chicago,2,36112 36113 36110 36111 36116 36114 36115 3610...,1840008353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12599,WY,Big Piney,Taylor Ave,83113.0,4.0,2.0,1820.0,112.637363,0.1600,189800.0,...,205000.0,450,342.1,shape,False,True,America/Denver,3,83113,1840020061
12600,WY,Daniel,Booth Dr,83115.0,1.0,1.0,1008.0,337.202381,10.0600,307500.0,...,339900.0,158,12.0,shape,False,False,America/Denver,3,83115,1840018659
12601,WY,Shoshoni,Pine St,82649.0,6.0,3.0,3294.0,74.377656,0.3200,236800.0,...,245000.0,640,69.0,shape,False,True,America/Denver,3,82649,1840022355
12602,WY,Mills,N 4th Ave,82604.0,4.0,2.0,2120.0,90.094340,0.3100,184600.0,...,191000.0,4218,319.8,shape,False,True,America/Denver,3,82644 82604 82601,1840022359
